In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

## Imports and Library Setup

In [ ]:
import { buildParser } from '@lezer/generator';
import { Tree, TreeCursor } from '@lezer/common';
import { LRParser } from '@lezer/lr';
import { RecursiveSet, RecursiveMap, Tuple } from "recursive-set";

# A Symbolic Calculator with Lezer \& RecursiveSet

This notebook demonstrates the implementation of a **symbolic calculator** that parses, analyzes, and evaluates assignments and arithmetic expressions.

The distinctive feature of this implementation is the use of **value semantics** for the Abstract Syntax Tree (AST) and the environment. Instead of ordinary JavaScript objects, we utilize the `recursive-set` library to enable structural equality and hash-based lookup.

## Language Grammar

The grammar for our calculator language is formally defined as follows:

$$
\begin{array}{lcl}
\texttt{stmnt} & \rightarrow & \texttt{IDENTIFIER}\; \texttt{':='}\; \texttt{expr}\; \texttt{';'} \\
& \mid & \texttt{expr}\; \texttt{';'} \\[0.3cm]
\texttt{expr} & \rightarrow & \texttt{expr}\; \texttt{'+'}\; \texttt{product} \\
& \mid & \texttt{expr}\; \texttt{'-'}\; \texttt{product} \\
& \mid & \texttt{product} \\[0.3cm]
\texttt{product} & \rightarrow & \texttt{product}\; \texttt{'*'}\; \texttt{factor} \\
& \mid & \texttt{product}\; \texttt{'/'}\; \texttt{factor} \\
& \mid & \texttt{factor} \\[0.3cm]
\texttt{factor} & \rightarrow & \texttt{'('}\; \texttt{expr}\; \texttt{')'} \\
& \mid & \texttt{'+'}\; \texttt{factor} \\
& \mid & \texttt{'-'}\; \texttt{factor} \\
& \mid & \texttt{NUMBER} \\
& \mid & \texttt{IDENTIFIER}
\end{array}
$$

This grammar ensures proper operator precedence (multiplication and division before addition and subtraction) through its hierarchical structure. Left recursion in the $\texttt{expr}$ and $\texttt{product}$ rules guarantees left-associative evaluation.

## Architecture \& Design Philosophy

The interpreter pipeline consists of four distinct stages:

1. **Grammar Definition:** We declaratively specify the language syntax using **Lezer**.
2. **Runtime Parsing:** `buildParser` generates an efficient LR parser from the grammar.
3. **AST Transformation (Tuple-based):** The flat Lezer tree is transformed into a typed AST where all nodes inherit from `Tuple`.
4. **Semantic Analysis \& Evaluation:** We employ `RecursiveSet` for variable analysis and `RecursiveMap` as a variable store.

### Why Tuple and RecursiveSet?

In classical AST implementations, two nodes `new BinOp("+", 1, 2)` and `new BinOp("+", 1, 2)` are **not equal** under JavaScript's default reference equality. By inheriting from `Tuple`, we obtain **structural equality**:

- **Identity:** Two AST nodes with identical content are logically equivalent
- **Hashing:** AST nodes can serve as keys in maps or elements in sets
- **Debugging:** `Tuple` automatically provides clean string representations (e.g., `(10 * 2)`)


## Domain Types (The AST)

We define the language constructs as classes inheriting from `Tuple`, ensuring type safety and immutability.

In [ ]:
type Operator = "+" | "-" | "*" | "/";

class Num extends Tuple<["num", number]> {
    constructor(val: number) {
        super("num", val);
    }
    get value(): number {
        return this.get(1);
    }

    toString() {
        return String(this.value);
    }
}

class Var extends Tuple<["var", string]> {
    constructor(name: string) {
        super("var", name);
    }
    get name(): string {
        return this.get(1);
    }

    toString() {
        return this.name;
    }
}

class BinOp<L extends Expr = Expr, R extends Expr = Expr> extends Tuple<
    [Operator, L, R]
> {
    constructor(op: Operator, left: L, right: R) {
        super(op, left, right);
    }

    get op(): Operator {
        return this.get(0);
    }
    get left(): L {
        return this.get(1);
    }
    get right(): R {
        return this.get(2);
    }

    toString() {
        return `(${this.left} ${this.op} ${this.right})`;
    }
}

class Assign extends Tuple<["assign", string, Expr]> {
    constructor(name: string, expr: Expr) {
        super("assign", name, expr);
    }

    get name(): string {
        return this.get(1);
    }
    get expr(): Expr {
        return this.get(2);
    }

    toString() {
        return `${this.name} := ${this.expr};`;
    }
}

class ExprStmnt extends Tuple<["expr_stmnt", Expr]> {
    constructor(expr: Expr) {
        super("expr_stmnt", expr);
    }

    get expr(): Expr {
        return this.get(1);
    }

    toString() {
        return `${this.expr};`;
    }
}

type Expr = Num | Var | BinOp;
type Stmnt = Assign | ExprStmnt;
type Env = RecursiveMap<string, number>;

function isOperator(s: string): s is Operator {
    return ["+", "-", "*", "/"].includes(s);
}

function isExpr(node: any): node is Expr {
    return node instanceof Num || node instanceof Var || node instanceof BinOp;
}

function ensureExpr(node: any): Expr {
    if (isExpr(node)) return node;
    throw new Error(
        `Erwarte Expr, erhielt: ${node?.constructor?.name ?? node}`,
    );
}

**Type Hierarchy:**

$$
\begin{array}{lcl}
\texttt{Expr} & ::= & \texttt{Num} \mid \texttt{Var} \mid \texttt{BinOp} \\
\texttt{Stmnt} & ::= & \texttt{Assign} \mid \texttt{ExprStmnt}
\end{array}
$$

## Grammar Definition (Lezer)

The grammar is specified declaratively using Lezer's syntax. Uppercase rule names (e.g., `Expr`, `Factor`) ensure that Lezer creates dedicated nodes in the parse tree, simplifying subsequent traversal.

- **Left Recursion Resolution:** Achieved through repetition patterns `Product ((Plus | Minus) Product)*`
- **Precedence Rules:** Multiplication/division precedence is enforced through nesting of `Expr` (additive operations) and `Product` (multiplicative operations)

In [ ]:
const grammarDefinition = `
@top Program { statement+ }

statement {
  Assignment { Identifier ":=" Expr ";" } |
  ExpressionStatement { Expr ";" }
}

Expr {
  Product ((Plus | Minus) Product)*
}

Product {
  Factor ((Mul | Div) Factor)*
}

Factor {
  ParenExpr { "(" Expr ")" } |
  UnaryExpr { (Plus | Minus) Factor } |
  Number |
  Identifier
}

@tokens {
  Number { (("0" | $[1-9] $[0-9]*) ("." $[0-9]*)?) }
  Identifier { $[a-zA-Z] $[a-zA-Z0-9_]* }
  Plus { "+" }
  Minus { "-" }
  Mul { "*" }
  Div { "/" }
  space { $[ \t\r]+ }
  "(" ")" ";" ":="
}

@skip { space }
`;

const parser: LRParser = buildParser(grammarDefinition);

### Tokenization Helper

In Lezer, tokenization is integrated with parsing. For debugging purposes, `tokenizeCalc` extracts tokens by traversing the parse tree and filtering leaf nodes.

In [ ]:
function tokenizeCalc(input: string): string[] {
    const tree = parser.parse(input);
    const cursor = tree.cursor();
    const tokens: string[] = [];
    const structuralNodes = new Set([
        "Program",
        "statement",
        "Assignment",
        "ExpressionStatement",
        "Expr",
        "Product",
        "Factor",
        "UnaryExpr",
        "ParenExpr",
    ]);
    do {
        if (!structuralNodes.has(cursor.name)) {
            const content = input.slice(cursor.from, cursor.to);
            let name = cursor.name;
            if (name === ":=") name = "Assign";
            if (name === ";") name = "Semicolon";

            tokens.push(`${name}('${content}')`);
        }
    } while (cursor.next());
    return tokens;
}


Example: 

In [ ]:
console.log(tokenizeCalc("x := 3 + 4 * 2;"));

## Parser Logic (lezerToAST)

The function `lezerToAST` transforms the efficient but flat Lezer tree into our rich `Tuple`-based structure.

**Key Features:**

- **Robust Navigation:** We search for child nodes dynamically by name rather than relying on fixed index positions, making the parser resilient to minor grammar changes
- **Type Guards:** We use `ensureExpr` and `isExpr` to ensure at runtime that we don't receive statements where expressions are expected
- **Error Handling:** Syntax errors (nodes marked `⚠`) are immediately caught and reported

In [ ]:
function lezerToAST(cursor: TreeCursor, source: string): Stmnt | Expr {
    const name = cursor.name;
    const text = source.slice(cursor.from, cursor.to);

    if (name === "⚠") throw new Error(`Syntax-Fehler nahe: '${text}'`);

    switch (name) {
        case "Assignment": {
            let varName: string | null = null;
            let exprNode: Expr | null = null;

            if (cursor.firstChild()) {
                do {
                    if (cursor.name === "Identifier") {
                        varName = source.slice(cursor.from, cursor.to);
                    } else if (cursor.name === "Expr") {
                        exprNode = ensureExpr(lezerToAST(cursor, source));
                    }
                } while (cursor.nextSibling());
                cursor.parent();
            }

            if (!varName) throw new Error("Assignment: Identifier fehlt");
            if (!exprNode) throw new Error("Assignment: Expression fehlt");

            return new Assign(varName, exprNode);
        }

        case "ExpressionStatement": {
            let exprNode: Expr | null = null;
            if (cursor.firstChild()) {
                do {
                    if (cursor.name === "Expr") {
                        exprNode = ensureExpr(lezerToAST(cursor, source));
                        break;
                    }
                } while (cursor.nextSibling());
                cursor.parent();
            }

            if (!exprNode) throw new Error("Leeres Statement");
            return new ExprStmnt(exprNode);
        }

        case "Expr":
            return parseLeftAssociative(cursor, source, ["+", "-"]);

        case "Product":
            return parseLeftAssociative(cursor, source, ["*", "/"]);

        case "UnaryExpr": {
            let op: Operator = "+";
            let factor: Expr | null = null;

            if (cursor.firstChild()) {
                do {
                    const t = source.slice(cursor.from, cursor.to);
                    if (isOperator(t)) {
                        op = t;
                    } else {
                        factor = ensureExpr(lezerToAST(cursor, source));
                    }
                } while (cursor.nextSibling());
                cursor.parent();
            }

            if (!factor) throw new Error("UnaryExpr ohne Wert");

            if (op === "-") {
                return new BinOp("*", new Num(-1), factor);
            }
            return factor;
        }

        case "ParenExpr": {
            let inner: Expr | null = null;
            if (cursor.firstChild()) {
                do {
                    if (cursor.name === "Expr") {
                        inner = ensureExpr(lezerToAST(cursor, source));
                        break;
                    }
                } while (cursor.nextSibling());
                cursor.parent();
            }
            return inner || new Num(0);
        }

        case "Number":
            return new Num(parseFloat(text));

        case "Identifier":
            return new Var(text);

        default:
            if (cursor.firstChild()) {
                const result = lezerToAST(cursor, source);
                cursor.parent();
                return result;
            }
            throw new Error(`Unerwarteter AST-Knoten: ${name}`);
    }
}

function parseLeftAssociative(
    cursor: TreeCursor,
    source: string,
    allowedOps: string[],
): Expr {
    const operands: Expr[] = [];
    const operators: Operator[] = [];

    if (cursor.firstChild()) {
        do {
            const text = source.slice(cursor.from, cursor.to);
            if (allowedOps.includes(text)) {
                if (isOperator(text)) operators.push(text);
            } else {
                operands.push(ensureExpr(lezerToAST(cursor, source)));
            }
        } while (cursor.nextSibling());
        cursor.parent();
    }

    if (operands.length === 0) return new Num(0);

    let result = operands[0];
    for (let i = 0; i < operators.length; i++) {
        const op = operators[i];
        const right = operands[i + 1];
        if (!right) throw new Error(`Operator '${op}' fehlt rechter Operand.`);

        result = new BinOp(op, result, right);
    }

    return result;
}

## Static Analysis

Instead of a simple `Record<string, number>`, we use `RecursiveMap` for the environment.

**Advantages:**

- **Efficiency:** Optimized hashing for string keys
- **Deterministic Output:** `env.toString()` produces sorted, deterministic output of the storage state

Additionally, we implement `collectVars`, which collects all used variables from an expression into a `RecursiveSet<string>`. This demonstrates how easily set operations can be performed with the library.

In [ ]:
function collectVars(node: Stmnt | Expr): RecursiveSet<string> {
    const vars = new RecursiveSet<string>();

    function visit(n: Stmnt | Expr): void {
        if (n instanceof Var) {
            vars.add(n.name);
        } else if (n instanceof Assign) {
            vars.add(n.name);
            visit(n.expr);
        } else if (n instanceof ExprStmnt) {
            visit(n.expr);
        } else if (n instanceof BinOp) {
            visit(n.left);
            visit(n.right);
        }
    }

    visit(node);
    return vars;
}

## Operational Semantics

The evaluation phase defines the semantic meaning of our AST nodes through mathematical operations on the abstract machine state.

### Expression Evaluation

The evaluation function $\mathcal{E}: \texttt{Expr} \times \texttt{Env} \to \mathbb{R}$ is defined recursively:

$$
\begin{array}{lcl}
\mathcal{E}(\texttt{Num}(n), \rho) & = & n \\[0.2cm]
\mathcal{E}(\texttt{Var}(x), \rho) & = & \rho(x) \\[0.2cm]
\mathcal{E}(\texttt{BinOp}(\oplus, e_1, e_2), \rho) & = & \mathcal{E}(e_1, \rho) \oplus \mathcal{E}(e_2, \rho)
\end{array}
$$

where $\rho: \texttt{String} \to \mathbb{R}$ represents the environment and $\oplus \in \{+, -, \times, \div\}$.

In [ ]:
function evalExpr(e: Expr, env: Env): number {
    if (e instanceof Num) {
        return e.value;
    } else if (e instanceof Var) {
        const val = env.get(e.name);
        if (val === undefined)
            throw new Error(`Variable '${e.name}' ist nicht definiert.`);
        return val;
    } else if (e instanceof BinOp) {
        const l = evalExpr(e.left, env);
        const r = evalExpr(e.right, env);
        switch (e.op) {
            case "+":
                return l + r;
            case "-":
                return l - r;
            case "*":
                return l * r;
            case "/":
                return l / r;
        }
    }
    throw new Error("Unbekannter Ausdruckstyp");
}

function execStmnt(s: Stmnt, env: Env): number | undefined {
    if (s instanceof Assign) {
        const val = evalExpr(s.expr, env);
        env.set(s.name, val);
        return val;
    } else if (s instanceof ExprStmnt) {
        return evalExpr(s.expr, env);
    }
    return undefined;
}

### Statement Execution

The execution function $\mathcal{S}: \texttt{Stmnt} \times \texttt{Env} \to \mathbb{R} \times \texttt{Env}$ produces both a result and an updated environment:

$$
\begin{array}{lcl}
\mathcal{S}(\texttt{Assign}(x, e), \rho) & = & \langle v, \rho[x \mapsto v] \rangle \quad \text{where } v = \mathcal{E}(e, \rho) \\[0.2cm]
\mathcal{S}(\texttt{ExprStmnt}(e), \rho) & = & \langle \mathcal{E}(e, \rho), \rho \rangle
\end{array}
$$

In [ ]:
function parseCalc(input: string): Stmnt {
    const tree = parser.parse(input);
    const cursor = tree.cursor();

    if (!cursor.firstChild()) throw new Error("Leeres Programm");
    if (cursor.name === "Program") {
        if (!cursor.firstChild()) throw new Error("Leeres Programm (Body)");
    }

    const ast = lezerToAST(cursor, input);
    if (isExpr(ast)) throw new Error("Erwarte Statement");
    return ast as Stmnt;
}

Set `inputProgram`for the Calculator:

In [ ]:
const inputProgram = `
x := 10 + 5 * 2;
y := (x - 5) / 3;
z := -y * 2;
z + 100;
`;

## Main Execution Loop

The main function orchestrates the Read-Eval-Print Loop (REPL):

In [ ]:
const env = new RecursiveMap<string, number>();

const lines = inputProgram
    .split("\n")
    .map((l) => l.trim())
    .filter((l) => l.length > 0);

console.log("--- Calculation Start ---");

for (const line of lines) {
    try {
        const stmnt = parseCalc(line);
        const vars = collectVars(stmnt);
        const res = execStmnt(stmnt, env);
        console.log(
            `AST: ${stmnt.toString().padEnd(30)} | Vars: ${vars} | Result: ${res}`,
        );
    } catch (e) {
        console.error(
            `Error in '${line}':`,
            e instanceof Error ? e.message : e,
        );
    }
}

console.log("\n--- Final Environment ---");
console.log(env.toString());